In [1]:
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2

In [2]:
def create_data_model():
    data = {}
    data['distance_matrix'] = [
        [
            0, 548, 776, 696, 582, 274, 502, 194, 308, 194, 536, 502, 388, 354,
            468, 776, 662
        ],
        [
            548, 0, 684, 308, 194, 502, 730, 354, 696, 742, 1084, 594, 480, 674,
            1016, 868, 1210
        ],
        [
            776, 684, 0, 992, 878, 502, 274, 810, 468, 742, 400, 1278, 1164,
            1130, 788, 1552, 754
        ],
        [
            696, 308, 992, 0, 114, 650, 878, 502, 844, 890, 1232, 514, 628, 822,
            1164, 560, 1358
        ],
        [
            582, 194, 878, 114, 0, 536, 764, 388, 730, 776, 1118, 400, 514, 708,
            1050, 674, 1244
        ],
        [
            274, 502, 502, 650, 536, 0, 228, 308, 194, 240, 582, 776, 662, 628,
            514, 1050, 708
        ],
        [
            502, 730, 274, 878, 764, 228, 0, 536, 194, 468, 354, 1004, 890, 856,
            514, 1278, 480
        ],
        [
            194, 354, 810, 502, 388, 308, 536, 0, 342, 388, 730, 468, 354, 320,
            662, 742, 856
        ],
        [
            308, 696, 468, 844, 730, 194, 194, 342, 0, 274, 388, 810, 696, 662,
            320, 1084, 514
        ],
        [
            194, 742, 742, 890, 776, 240, 468, 388, 274, 0, 342, 536, 422, 388,
            274, 810, 468
        ],
        [
            536, 1084, 400, 1232, 1118, 582, 354, 730, 388, 342, 0, 878, 764,
            730, 388, 1152, 354
        ],
        [
            502, 594, 1278, 514, 400, 776, 1004, 468, 810, 536, 878, 0, 114,
            308, 650, 274, 844
        ],
        [
            388, 480, 1164, 628, 514, 662, 890, 354, 696, 422, 764, 114, 0, 194,
            536, 388, 730
        ],
        [
            354, 674, 1130, 822, 708, 628, 856, 320, 662, 388, 730, 308, 194, 0,
            342, 422, 536
        ],
        [
            468, 1016, 788, 1164, 1050, 514, 514, 662, 320, 274, 388, 650, 536,
            342, 0, 764, 194
        ],
        [
            776, 868, 1552, 560, 674, 1050, 1278, 742, 1084, 810, 1152, 274,
            388, 422, 764, 0, 798
        ],
        [
            662, 1210, 754, 1358, 1244, 708, 480, 856, 514, 468, 354, 844, 730,
            536, 194, 798, 0
        ],
    ]
    
    data['num_vehicles'] = 4
    data['depot'] = 0
    data['demands'] = [0, 1, 1, 2, 4, 2, 4, 8, 8, 1, 2, 1, 2, 4, 4, 8, 8]
    data['vehicle_capacities'] = [15, 15, 15, 15]
    data['pickups_deliveries'] = [
        [1, 6],
        [2, 10],
        [4, 3],
        [5, 9],
        [7, 8],
        [15, 11],
        [13, 12],
        [16, 14],
    ]
    data['time_windows'] = [
        (0, 5),   # depot
        (7, 12),  # 1
        (10, 15), # 2
        (16, 18), # 3
        (10, 13), # 4
        (0, 5),   # 5
        (5, 10),  # 6
        (0, 4),   # 7
        (5, 10),  # 8
        (0, 3),   # 9
        (10, 16), # 10
        (10, 15), # 11
        (0, 5),   # 12
        (5, 10),  # 13
        (7, 8),   # 14
        (10, 15), # 15
        (11, 15), # 16
    ]

    
    
    
    
    return data

In [6]:
def main():
    data = create_data_model()
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']), data['num_vehicles'], data['depot'] )
    routing = pywrapcp.RoutingModel(manager)
    
    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]
    
    
    def time_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_node)
        return data['time_windows'][from_node][to_node]
    
#     def demand_callback(from_index):
#         from_node = manager.IndexToNode(from_index)
#         return data['demands'][from_node]
    
    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
#     demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    
    
    
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)


        
    
    
#     routing.AddDimensionWithVehicleCapacity(
#         demand_callback_index,
#         0,
#         data['vehicle_capacities'],
#         True,
#         'Capacity'
#     )
    
    routing.AddDimension(
        transit_callback_index,
        0,
        3000,
        True,
        "Distance"
    )
    
    
    distance_dimension = routing.GetDimensionOrDie("Distance")
    distance_dimension.SetGlobalSpanCostCoefficient(100)

    
    for request in data['pickups_deliveries']:
        pickup_index = manager.NodeToIndex(request[0])
        delivery_index = manager.NodeToIndex(request[1])
        
        routing.AddPickupAndDelivery(pickup_index, delivery_index)
        
        routing.solver().Add(
            routing.VehicleVar(pickup_index) == routing.VehicleVar(delivery_index) 
        )
        
        routing.solver().Add(
            distance_dimension.CumulVar(pickup_index) <= distance_dimension.CumulVar(delivery_index)
        )
        
    
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
#     search_parameters.first_solution_strategy = (
#         routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION
#     )

    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    )
    
    search_parameters.time_limit.FromSeconds(1)
    solution = routing.SolveWithParameters(search_parameters)
    if solution:
        print_solution(data, manager, routing, solution)
    else:
        print("no solution")

In [7]:
def print_solution(data, manager, routing, solution):
    max_routing_distance = 0
    
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}: \n'.format(vehicle_id)
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += '{} -> '.format(manager.IndexToNode(index))
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id 
            )
        plan_output += '{}\n '.format(manager.IndexToNode(index))
        plan_output += 'Distance of the Route {}m \n'.format(route_distance)
        print(plan_output)
        max_routing_distance = max(route_distance, max_routing_distance)

    print('Maximum of the route distances: {}m'.format(max_routing_distance))

In [8]:
main()

Route for vehicle 0: 
0 -> 16 -> 14 -> 13 -> 12 -> 0
 Distance of the Route 1780m 

Route for vehicle 1: 
0 -> 5 -> 2 -> 10 -> 9 -> 0
 Distance of the Route 1712m 

Route for vehicle 2: 
0 -> 4 -> 3 -> 15 -> 11 -> 0
 Distance of the Route 2032m 

Route for vehicle 3: 
0 -> 7 -> 1 -> 6 -> 8 -> 0
 Distance of the Route 1780m 

Maximum of the route distances: 2032m
